# Coursework for MRI reconstruction (Autumn 2019)

In this tutorial, we provide the data loader to read and process the MRI data in order to ease the difficulty of training your network. By providing this, we hope you focus more on methodology development. Please feel free to change it to suit what you need.

Introduction  ------

In [1]:
#ALL IMPORTS
import h5py, os
from functions import transforms as T
from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from matplotlib import pyplot as plt
#cuda
import torch.optim as optim
device = 'cuda:0' if torch.cuda.is_available() else 'cpu' #'cpu'

In [2]:
##### UNET MODEL ######

class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, relu activation and dropout.
    """

    def __init__(self, in_chans, out_chans, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input.
            out_chans (int): Number of channels in the output.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = drop_prob

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1),
            nn.InstanceNorm2d(out_chans),
            nn.ReLU(),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1),
            nn.InstanceNorm2d(out_chans),
            nn.ReLU(),
            nn.Dropout2d(drop_prob)
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        return self.layers(input)

    def __repr__(self):
        return f'ConvBlock(in_chans={self.in_chans}, out_chans={self.out_chans}, ' \
            f'drop_prob={self.drop_prob})'


class UnetModel(nn.Module):
    """
    PyTorch implementation of a U-Net model.

    This is based on:
        Olaf Ronneberger, Philipp Fischer, and Thomas Brox. U-net: Convolutional networks
        for biomedical image segmentation. In International Conference on Medical image
        computing and computer-assisted intervention, pages 234–241. Springer, 2015.
    """

    def __init__(self, in_chans, out_chans, chans, num_pool_layers, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input to the U-Net model.
            out_chans (int): Number of channels in the output to the U-Net model.
            chans (int): Number of output channels of the first convolution layer.
            num_pool_layers (int): Number of down-sampling and up-sampling layers.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.chans = chans
        self.num_pool_layers = num_pool_layers
        self.drop_prob = drop_prob

        self.down_sample_layers = nn.ModuleList([ConvBlock(in_chans, chans, drop_prob)])
        ch = chans
        for i in range(num_pool_layers - 1):
            self.down_sample_layers += [ConvBlock(ch, ch * 2, drop_prob)]
            ch *= 2
        self.conv = ConvBlock(ch, ch, drop_prob)

        self.up_sample_layers = nn.ModuleList()
        for i in range(num_pool_layers - 1):
            self.up_sample_layers += [ConvBlock(ch * 2, ch // 2, drop_prob)]
            ch //= 2
        self.up_sample_layers += [ConvBlock(ch * 2, ch, drop_prob)]
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch, ch // 2, kernel_size=1),
            nn.Conv2d(ch // 2, out_chans, kernel_size=1),
            nn.Conv2d(out_chans, out_chans, kernel_size=1),
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        stack = []
        output = input
        # Apply down-sampling layers
        for layer in self.down_sample_layers:
            output = layer(output)
            stack.append(output)
            output = F.max_pool2d(output, kernel_size=2)

        output = self.conv(output)

        # Apply up-sampling layers
        for layer in self.up_sample_layers:
            output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False)
            output = torch.cat([output, stack.pop()], dim=1)
            output = layer(output)
        return self.conv2(output)

In [3]:
#pre-defined ultily functions used in creating the neural net

def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id  
    
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape

    # apply random mask
    shape = np.array(slice_kspace.shape)
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masks = mask.repeat(S, Ny, 1, ps)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)

    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
#     A = cropped_img_und.squeeze().detach().cpu()
#     C = cropped_gt.squeeze().squeeze().detach().cpu()
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
    
    volume_image_abs = T.complex_abs(img_gt) 
    cropped_gt = T.center_crop(volume_image_abs, [320, 320])
    cropped_gt = cropped_gt
        
    volume_image_abs = T.complex_abs(img_und) 
    cropped_img_und = T.center_crop(volume_image_abs, [320, 320])
    cropped_img_und = cropped_img_und
        
    return cropped_gt,cropped_img_und, norm
  #return img_gt.squeeze(0), img_und.squeeze(0), rawdata_und.squeeze(0), masks.squeeze(0), norm

class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list
        self.acceleration = acceleration
        self.center_fraction = center_fraction
        self.use_seed = use_seed

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        subject_id = self.data_list[idx]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)
    
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]

    for i in range(len(data_path)):

        data_list[train_and_val[i]] = []

        which_data_path = data_path[i]

        for fname in sorted(os.listdir(which_data_path)):

            subject_data_path = os.path.join(which_data_path, fname)

            if not os.path.isfile(subject_data_path): continue 

            with h5py.File(subject_data_path, 'r') as data:
                num_slice = data['kspace'].shape[0]

            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]

    return data_list  
   
    
from skimage.measure import compare_ssim  
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )  


def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

In [4]:

torch.manual_seed(42)

# Now we can create a model
model = UnetModel(
    in_chans = 1,
    out_chans = 1,
    chans = 32,
    num_pool_layers = 4,
    drop_prob = 0 ).to(device) # FirstModel().to(device)

# # we can also inspect its parameters
# print("Before training: \n", model.state_dict())

In [5]:
# #Set initial path for the fullset of training data
# file_path = '/data/local/NC2019MRI/train'

# # Shuffle sets, and split train_index into train and validation 80:20 respectively 
# numfiles_all = len(os.listdir(file_path))
# print (numfiles_all)
# indx = np.arange(numfiles_all)
# np.random.shuffle(indx)

# split_80_20 = 0.8 * numfiles_all
# split_80_20 = int(split_80_20)
# print(split_80_20)

# np.random.seed(42)

# train_inx = indx[:split_80_20]
# val_inx = indx[split_80_20:]

# print("train_inx",train_inx)
# print("val_inx",val_inx)


In [6]:

#     train_data_list = {}
#     train_data_list['val']= [0] * len(val_inx)
#     train_data_list['train'] = [0] * len(train_inx)
#     count = 0  
    
#     for fname in sorted(os.listdir(file_path)): 
#         subject_path = os.path.join(file_path, fname)
#         with h5py.File(subject_path,  "r") as hf:
#              total_num_slices = hf['kspace'].shape[0]

                
#         for i in range(len(train_inx)):
#             if (train_inx[i] == count):
#                 train_data_list['train'][i] = (file_path+ "/" +fname, fname, total_num_slices)  
#                 break
                
#         for k in range(len(val_inx)):
#             if (val_inx[k] == count):
#                 train_data_list['val'][k] = (file_path+ "/"+ fname, fname, total_num_slices)
#                 break
#         count = count +1

            
#     train_data_final ={}         
#     train_data_final['train'] = []  
#     train_data_final['val'] = []  
           
#     for i in range (len(train_data_list['train'])):
#         for slice in range (5,train_data_list['train'][i][2]):
#             train_data_final['train'].append((train_data_list['train'][i][1], train_data_list['train'][i][0], (slice)))
        

#     for i in range (len(train_data_list['val'])):
#         for slice in range (5,train_data_list['val'][i][2]):
#             train_data_final['val'].append((train_data_list['val'][i][1], train_data_list['val'][i][0], slice))

            

In [7]:
# print("VALIDATION SET")
# valcount = 0
# for i in range (len(train_data_final['val'])):
#     print(" count : ",valcount,"  ", train_data_final['val'][i])
#     valcount = valcount + 1
    
# print("")
# print("")
# print("")
# print("")

# train_count = 0    
# print("TRAIN SET")
# for i in range (len(train_data_final['train'])):
#     print(" count : ",train_count,"  ", train_data_final['train'][i])
#     train_count = train_count + 1


In [8]:
    data_path_train = '/data/local/NC2019MRI/train'
    data_path_val = '/data/local/NC2019MRI/train'
    data_list = load_data_path(data_path_train, data_path_val) # first load all file names, paths and slices.


In [9]:
######loads_train_data #######
if __name__ == '__main__':
    
    acc = 8
    cen_fract = 0.04
    seed = False # random masks for each slice 
    num_workers = 12 
    
    
    #
    train_dataset = MRIDataset(train_data_final['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=5, num_workers=num_workers) 
    
    validation_dataset = MRIDataset(train_data_final['val'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    validation_loader = DataLoader(validation_dataset, shuffle=True, batch_size=5, num_workers=num_workers)  

NameError: name 'train_data_final' is not defined

In [1]:
#set training
%matplotlib inline 
print("HELOLOOO")
# set learning rate

#lr = 1e-4
# lr = 1e-3
# lr = 1e-2
lr = 1e-1

#set number of epoches, i.e., number of times we iterate through the training set
epoches = 50

# # We use mean square error (MSELoss)
loss_fn = nn.MSELoss(reduction='mean').to(device)

# Optimisers
#optimiser = torch.optim.RMSprop(model.parameters(), lr=lr)
optimiser = optim.SGD(model.parameters(), lr=lr)





#Lists for plotting val/train SSIM/Loss/prEpoch
train_av_epoch_ssim_list = []
train_av_epoch_loss_list = []

val_av_epoch_ssim_list = []
val_av_epoch_loss_list = []

epoch_count = 0

#Model training
for epoch in range(epoches):
   
    model.train()
    
    train_total_loss = 0
    train_total_ssim = 0
    train_iterations= 0
    
    val_total_loss = 0
    val_total_ssim = 0
    val_iterations= 0
    
    #Training
    for data in train_loader:

        # set the model to training mode
        img_gt, img_und, _ = data
        
        #Reconstruction/prediction from undersampled data
        img_gt = img_gt.to(device) 
        img_und.to(device) 
        optimiser.zero_grad()
        print(train_iterations, '!!!!!size:',img_und.size(), img_gt.size())
        y_pred = model(img_und.to(device))
        
                #visualise 
        # 0 = UNDERSAMPLED. 1 = PREDICTION. 2 = GROUND_TRUTH
        #if train_iterations % 10 == 1:
        A = img_und.squeeze().squeeze().numpy()[0:1,:,:]
        B = y_pred.squeeze().squeeze().squeeze(0).detach().cpu()
        
        B = y_pred.squeeze().data.cpu().numpy()
        B = B[0:1, :, :]
        #print(B.shape)
        
        C = img_gt.squeeze().squeeze().squeeze(0).cpu().numpy()[0:1,:,:]
        all_imgs = np.concatenate([A,B,C], axis=0)

        #Visualise
        #show_slices(all_imgs, [0,1,2], cmap='gray')
        #plt.pause(1)

        
        #calculate the loss from training data
        loss = loss_fn(img_gt.to(device), y_pred)
        loss.backward() 
        optimiser.step()
        
        #calculate total loss for epoch to generate average loss
        train_total_loss = train_total_loss + loss.item()
        
        # calculate total ssim value for epoch to generate average ssim
        ssim_value = ssim(C,B)
        train_total_ssim = train_total_ssim + ssim_value
     

        #count
        train_iterations = train_iterations +1
        
        
    train_av_ssim = train_total_ssim/train_iterations
    train_av_loss = train_total_loss/train_iterations
    
    train_av_epoch_ssim_list.append(train_av_ssim)
    train_av_epoch_loss_list.append(train_av_loss)
    
    
#     #Validation 
#     with torch.no_grad():
#         #set to evaluation mode
#         model.eval()
#         for data in validation_loader:

#             #from data loader
#             img_gt, img_und, _ = data
            
#             #Reconstruction/prediction from undersampled data
#             img_gt = img_gt.to(device) 
#             img_und.to(device) 
#             optimiser.zero_grad()
#             print('!!!!!size:',img_und.size(), img_gt.size())
#             y_pred = model(img_und.to(device))
            
#             A = img_und.squeeze().squeeze().numpy()[0:1,:,:]
#             B = y_pred.squeeze().squeeze().squeeze(0).detach().cpu()
        
#             B = y_pred.squeeze().data.cpu().numpy()
#             B = B[0:1, :, :]
            
#         #print(B.shape)
        
#             C = img_gt.squeeze().squeeze().squeeze(0).cpu().numpy()[0:1,:,:]
#             all_imgs = np.concatenate([A,B,C], axis=0)
            
            
#              #calculate the loss from validation data
#             val_loss = loss_fn(img_gt.to(device), y_pred)
#             optimiser.step()
            
#             #calculate total loss for epoch to generate average loss
#             val_total_loss = val_total_loss + loss.item()
        
        
# #             A = img_und.unsqueeze(1).n
# #             B = y_pred.squeeze().squeeze().detach().cpu().numpy()
# #             C = img_gt.squeeze().squeeze().cpu().numpy()
# #             #print(A.shape, B.shape, C.shape)
            
#             #all_imgs = torch.stack([A.squeeze(),B,C], dim=0)
#             #print("1")
#            # show_slices(all_imgs, [0,1,2], cmap='gray')
#             #plt.pause(1)
#             #print("2")
#             #calculate total ssim value for epoch to generate average ssim
#             ssim_value = ssim(C,B)
#             val_total_ssim = val_total_ssim + ssim_value
            
#             #count
#             val_iterations = val_iterations + 1
            
#     val_av_ssim = val_total_ssim/val_iterations
#     val_av_loss = val_total_loss/val_iterations
    
#     val_av_epoch_ssim_list.append(val_av_ssim)
#     val_av_epoch_loss_list.append(val_av_loss)
    
#     epoch_count = epoch_count + 1
    print("epoch_count ", epoch_count)

print("epoch: ", epoch_count,"train_av_epoch_ssim_list :", train_av_epoch_ssim_list)
print("epoch: ", epoch_count,"train_av_epoch_loss_list :", train_av_epoch_loss_list)
#print("epoch: ", epoch_count,"val_av_epoch_ssim_list :", val_av_epoch_ssim_list)
#print("epoch: ", epoch_count,"val_av_epoch_loss_list :", val_av_epoch_loss_list)


HELOLOOO


NameError: name 'nn' is not defined

In [ ]:
#VALIDATION VS TRAINING SSIM COMPARISON
plt.plot(range(len(train_av_epoch_ssim_list)), train_av_epoch_ssim_list,'r', label = "Av Train SSIM")
plt.plot(range(len(val_av_epoch_ssim_list)), val_av_epoch_ssim_list,'b', label = "Av Val SSIM")
plt.xlabel("epochs")
plt.ylabel("Av SSIM")
plt.legend()
plt.show()

In [ ]:
#VALIDATION VS TRAINING LOSS COMPARISON
plt.plot(range(len(train_av_epoch_loss_list)), train_av_epoch_loss_list,'r', label = "Av Training loss")
plt.plot(range(len(val_av_epoch_loss_list)), val_av_epoch_loss_list,'b', label = "Av Val loss")
plt.xlabel("Epochs")

plt.legend()
plt.show()

In [ ]:
torch.save(model,"unet_model_save_50ep_sgd_le1_b5_L1.pkl")

In [ ]:
#x = train_av_epoch_loss_list
#x = val_av_epoch_loss_list
#x = train_av_epoch_ssim_list
x = val_av_epoch_ssim_list

total = 0
for i in range(len(x)):
    total = total +x[i]
    av = total/(len(x))
print(av)


# Introduction

The MRI data

How it generated

Ksapce

How converted to a normal image

task and aim

# Design

## U-net

![unet.png](./unet.png)
In 2015, OlafRonneberger and others proposed a U-net network structure. The U-net network is a semantic segmentation network based on FCN, which is suitable for medical image segmentation.


The entire U-Net network structure is shown in the figure above, similar to a large U letter: first convolution + Pooling down-sampling; then deconvolution for up-sampling, low-level feature map before the crop, and fusion; then up-sampling again. Repeat this process until you get a feature map with an output of 388x388x2, and finally, get the output segment map through soft-max. Overall, it is very similar to the FCN idea. (If not necessary, delete them) Unet's original intention is to solve the problems of biomedical images. Since the effect is really good, it has been widely used in various directions of semantic segmentation, such as satellite image segmentation and industrial defect detection.
[Ronneberger O, Fischer P, Brox T, U-net: Convolutional networks for biomedical image segmentation. International Conference on Medical image computing and computer-assisted intervention, 2015 ]

[Wenjun Y, Yuanyuan W, Shengjia G, The Domain Shift Problem of Medical Image Segmentation and Vendor-Adaptation by Unet-GAN. International Conference on Medical Image Computing and Computer-Assisted Intervention, 2019 ]

### Advantages
• Support for training models with a small amount of data 

• Get higher segmentation accuracy by classifying each pixel 

• Fast segmentation with trained models 


## Gan

GAN is composed of a generator and a discriminator. The purpose of the generator is to generate fake targets in an attempt to completely deceive the discriminator. The discriminator improves its discrimination ability by learning the true target and the false target, so as not to let the false target fool itself. The two evolved and played against each other. One side evolved and the other lost. Finally, the evolution stopped until the false target was very similar to the true target. The main flow is similar to the figure above. First of all, there is a generation generator that can generate some very poor pictures, and then there is a generation discriminator that can accurately classify the generated picture with the real picture, and then repeat the above action(If not necessary, delete them) Compared with all other models, GAN can produce clearer and true samples. GAN uses an unsupervised learning method for training, which can be widely used in the fields of unsupervised learning and semi-supervised learning. However, because of the unsupervised GAN, in the generation process, it will generate some weird pictures according to its own wishes for a long time. What’s worse, this picture can still get a high score.

[ Sebastian N, Botond C, Ryota T, f-GAN: Training Generative Neural Samplers using Variational Divergence Minimization.  Advances in Neural Information Processing Systems, 2016 ]

## Conclusion

In this experiment, we chose U-net as our training model due to we have seen that U-net has achieved good results in this field in many previous studies, while GAN has fewer applications in medical imaging. Also, U-net has a fast processing speed and a higher segmentation accuracy which is beneficial to our experiment.

# Implementation

# Experiments

## Batch 

Batch is the number of samples selected for a single training session. Its size affects the degree and speed of optimization of the model. At the same time, it directly affects the GPU memory usage. When chose the best batch size, it can improve memory utilization through parallelization by enabling GPU to run at full capacity to improve training speed. Also, the number of iterations of a single epoch is reduced and the adjustment of parameters is slow, which more epochs are required to achieve the same recognition accuracy. Most importantly, the proper batch will improve the gradient descent.
[ Alfredo C, Adam P, Eugenio C, An Analysis of Deep Neural Network Models for Practical Applications. arXiv preprint arXiv:1605.07678, 2016]

We calculated the average loss function and SSIM of both training data and validation data set with the batch from 3 to 6 when epochs is 0 to 50. We can easily conclude from the table that the SSIM and is better when the Batch is 5 as we list the result as below.

|    Batch Size    | Average Training Data Loss   |    Average Validation Data Loss    | Average Training Data SSIM |    Average Validation Data SSIM    |
| ---------- | --- | --- | --- | --- |
| 3 |  0.00816| 0.00859| 0.47535 | 0.50365|
| 4 |  0.00808| 0.00784| 0.48208 | 0.45985|
| 5 |  0.00891| 0.00913| 0.45697 | 0.53715|
| 6 |  0.00872| 0.00890| 0.46395 | 0.49930|

When the batch is 5, the obtained data curve is shown below. Both the average SSIM and the SSIM at 50 epochs of the training data set is around 0.46. Since the SSIM of validation data set is more vibrate and with an average of 0.53 but can reach about 0.57 at 50 epochs, which is significant. 

As for the training data loss function, statistics tend to descend steadily. Although validation data loss contains a lot of noise, the overall trend is also decreasing.

![Batch5Loss.png](./Batch5Loss.png)
![Batch5SSIM.png](./Batch5SSIM.png)


## Learning Rate

The learning rate determines how far the weights move in the gradient direction in a batch. Choosing a suitable learning rate requires constant experimentation and adjustment.

When the learning rate is very small,  the training will become reliable, that is, the gradient will gradually approach the minimum value. The calculated loss will become smaller as well as the cost is that the training time will take longer. When the learning rate is very large, the training will ignore the minimum value, showing that the loss value continuously fluctuates. In the most severe cases, it may never reach the minimum value, or even jump out of this range and fall into another sinking area. [ Robert A.Jacobs Increased rates of convergence through learning rate adaptation. Neural Networks, 1988]

We calculated the average loss function and SSIM of training data set with the batch size of 5 and epochs is 0 to 50. The following figures are examples of the average loss of different learning rate. The average loss drops rapidly around the first five epochs, and then it tends to decrease steadily. When enhancing the learning rate from 1e-1 to 1e-5, the loss in of the same epoch increases, and its overall trend remains unchanged. 
![LR1Loss.png](./LR1Loss.png)
![LR4Loss.png](./LR4Loss.png)




The following are the results of average SSIM. It can be seen from the figures that average SSIM increased rapidly around the first five epochs, followed by an upward trend, and accompanied by fluctuations of a certain intensity. When learning rate increasing, the subsequent fluctuation of the SSIM value weakened, and the results obtained gradually became more stable.
![LR1SSIM.png](./LR1SSIM.png)
![LR4SSIM.png](./LR4SSIM.png)


The following table is a summary of the results of the above. In the obtained results, the learning rate of 1e-1 to le-4 all conform to the above analysis, but the hyperparameter of 1e-5 do not correspond to the above rule. This is because after the learning rate increases, if epoch keeps at the same value, then the number of samples received is not enough to reflect the overall changing trend of average loss and average SSIM.

|    Learning Rate    | Average Training Data Loss   | Average Training Data SSIM |
| ---------- | --- | --- |
| 1e-1 |  0.00825| 0.47871| 
| 1e-2 |  0.00808| 0.48208| 
| 1e-3 |  0.01674| 0.35583|
| 1e-4 |  0.03098| 0.22719|
| 1e-5 |  0.01797| 0.10929|

![LR5Loss.png](./LR5Loss.png)
![LR5SSIM.png](./LR5SSIM.png)

The above analysis all based on 4-folders, we calculated with 8 times acceleration rate. We change the epoch to 120 and the learning rate is le-1 to achieve a better result. The leaning of the two values is almost the same as that in the case of 4-folders, except that the fluctuation of SSIM is slightly more obvious due to the increase of epoch. From the table below, its average loss is more miniature and the average SSIM is higher.

|    Acceleration Rate    | Average Training Data Loss   | Average Training Data SSIM |
| ---------- | --- | --- |
| 4 times |  0.00825| 0.47871| 
| 8 times |  0.00437| 0.59028| 


## Loss Function
 The predicted value obtained after using the model for training in the neural network needs to be compared with the ground truth so that the model can self-corrects until the difference becomes really small. With different loss functions, the influence on the model will differ too. We need to choose the most suitable function to get the most ideal model. We decided to use L1 as the orimary loss functions as: 

#### L1 Loss

$${
  loss(x,y)= \sum_{i=1}^{n} (x-y)^2 
 }$$

L1 Loss Function is used to minimize the error which is the sum of the all the absolute differences between the true value $x$ and the predicted value $y$. This Function is not affected by the outliers or remove the outliers. 
[Jonathan T. Barron, A General and Adaptive Robust Loss Function. The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2019]

This function can be used in data set may contain outliers.

## Optimizer

The main purpose of training a neural network is to find suitable parameters and obtain a relatively small loss function. The process of solving this problem is called optimization. The algorithm used to solve this problem is called optimizer. Among various optimizers, we choose three of them as listed below into our account. These three are basicly improved versions based on other optimizers, thus can achieve better result.  


#### Stochastic Gradient Descent (SGD)


Stochastic gradient descent (SGD)  computes the gradient of the cost function with respect to the parameters $\theta$. What's more, the whole training dataset delivers a parameter update for each training example $x^{(i)}$ and label $y^{(i)}$. It prevents redundant computations for large datasets and usually processes relatively fast. However, since SGD conducts frequent updates with high variance, it can cause the objective function to fluctuate rigorously. Also, when facing local optimal where curves are much more steeply in one dimension than in another. SGD oscillates across the area and makes slow progress. This leads to RMSprop.  [ Bottou L, Large-Scale Machine Learning with Stochastic Gradient Descent. Proceedings of COMPSTAT, 2010]

$$\theta = \theta - \eta \cdot \nabla_\theta J ( \theta ; x^{(i)} ; y^{(i)} ) $$

 In the experiments above, SGD it the default optimizer we use, it can be seen that the result is very good.


#### RMSprop

For the learning rate of models, traditional optimizer either set the learning rate to a constant or adjust the learning rate based training results. All ignore the potential of the learning rate. 

We can consider RMSprop as an adaptation of the rprop algorithm to learning with mini-batch. This was the original motivation for developing the algorithm. It performs low learning rates on parameters related to frequently occurring features and performs high learning rates to obtain parameters related to less commonly used features. Therefore, it is very suitable for dealing with sparse data. In this way, it can greatly improve the robustness of SGD and solve the problem of the sharp drop in learning rate. It is assumed that during the $t$ iteration, each formula is as follows:

[Geoffrey Hinton Neural Networks for machine learning nline course. https://www.coursera.org/learn/neural-networks/home/welcome]


$${
        s_{dw}= \beta s_{dw} + (1 - \beta)d W^2 \\
        s_{db}= \beta s_{db} + (1 - \beta)d b^2 \\
        W = W - \alpha \frac{dW}{\sqrt{s_{dw}} + \varepsilon} \\
        b = b - \alpha \frac{db}{\sqrt{s_{dw}} + \varepsilon}
 }$$


In the above formula, $s_{dw}$ and $s_{db}$ are the gradient momentum accumulated by the loss function during the first $t-1$ iterations,.Respectively,  $\beta$ is an index of gradient accumulation. In order to prevent the denominator from being zero, a very small value $\varepsilon$ is used for smoothing which generally take 1e-8.

The RMSprop algorithm calculates the differential squared weighted average of the gradient. This method is beneficial to eliminate the direction of large swing amplitude and is used to correct the swing amplitude so that the swing amplitude in each dimension is smaller. On the other hand, it also makes the network function converge faster.

We calculated the average SSIM obtained after the data set uses RMSprop with 50 epochs and displayed in the figure below. It can be seen from the figure that the data used in our training fluctuates rigorously and the result is not clear, so we do not use RMSprop in this experiment.

![RMSprop.png](./RMSprop.png)

#### Adaptive Moment Estimation (Adam)


Adaptive Moment Estimation (Adam) is a method that computes adaptive learning rates for each parameter. In addition to collecting past squared gradients' exponentially decaying average $v_t$, Adam also keeps an exponentially decaying average of past gradients $m_t$ which similar to momentum. In this way, it can flat minima in the error surface. We compute the $v_t$ and $m_t$ as follows:  [Bottou, L, The Tradeoffs of Large Scale Learning.  Optimization for Machine Learning, 2012]

$${
     m_t = \beta_1 m_{t-1} + ( 1- \beta_1) g_t \\
     v_t = \beta_2 v_{t-1} + ( 1- \beta_2) g_t^2
 }$$
 
If $m_t$ and $v_t$ are initialized as zero vectors, they will be biased towards 0, so a bias correction appear. It can offset these deviations by calculating the updated $m_t$ and $v_t$ as follows:

$${
     \hat m_t = \frac{m_t}{1 - \beta_1^t} \\
     \hat v_t = \frac{v_t}{1 - \beta_2^t}
 }$$

and last upated Adam is:

$${
        \theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat v_t} + \varepsilon} \hat m_t
 }$$
 

where $\beta_1$ and $\beta_2$ is often to set close to $1$

Adam is generally considered to be quite robust in the choice of hyperparameters, although the learning rate sometimes needs to be modified to default.

#### SGD versus Adam

The following table compares the average loss as well as the average SSIM of SGD and Adam when taking five batches and 50 epochs. It can be noticed directly that Adam performs better, so we use Adam as our optimizer.

|   Optimizer   | Average Loss   | Average SSIM | 
| ---------- | --- | --- |
| SGD |  0.00891| 0.45697| 
| Adam |  0.00776| 0.49972| 




Ref：https://ruder.io/optimizing-gradient-descent/

# Conclusions

# Contribution

# Reference